## **V. How Subroutines are Placed in Procedures**
Learn how to write reusable code using procedures. You'll see how the stack saves registers and passes parameters, how procedures are structured, and how assembly handles recursion. Special tools like INVOKE will be introduced for efficiently calling procedures.

## Topics Covered
A. How stack operations work
1. Using the stack to preserve register values
2. Using the stack to pass parameters

B. Procedures are defined using specific syntax

C. The call to a procedure

D. What to do when procedures are in an external library

E. What is a stack frame and how is it used

F. Performing recursion performing recursion performing recursion

G. INVOKE, the powerful procedure tool available in 32-bit x86 assembler

### **A. How Stack Operations Work**

---

### 📚 What is the Stack?

The **stack** in assembly language is a **LIFO (Last-In, First-Out)** memory structure used for:

* Saving temporary data (like registers)
* Passing function parameters
* Storing return addresses

The stack **grows downward** in memory on x86 architectures.

---

### 📌 Stack Instructions

| Instruction | Description                           |
| ----------- | ------------------------------------- |
| `push`      | Pushes a value onto the stack         |
| `pop`       | Pops (removes) a value from the stack |
| `call`      | Pushes return address and jumps       |
| `ret`       | Pops return address and jumps to it   |

---

### 🔹 1. Using the Stack to Preserve Register Values

Before calling a subroutine, it's good practice to save the current values of registers that will be used.

```nasm
push eax       ; Save value of eax
push ebx       ; Save value of ebx

; --- Subroutine Logic ---
mov eax, 10
mov ebx, 20

pop ebx        ; Restore original ebx
pop eax        ; Restore original eax
```

**Why?** So that the subroutine doesn't overwrite values the main program still needs.

---

### 🔹 2. Using the Stack to Pass Parameters

You can pass arguments to a procedure using the stack (especially in 32-bit assembly):

```nasm
; Passing 5 and 3 as arguments (in reverse order)
push 3
push 5
call AddNumbers
```

In the subroutine, the values are accessed relative to the `ebp` (base pointer), once a **stack frame** is set up.

---

### ⚙️ How This Works in Memory

Each `push` decreases the `esp` (stack pointer) and stores the value at the new address.
Each `pop` increases `esp` and loads the value back into the target register.

---

### 🧪 Example: Stack Operation Demo

```nasm
section .text
global _start

_start:
    mov eax, 7
    mov ebx, 2

    push eax
    push ebx

    pop ecx    ; Should be 2
    pop edx    ; Should be 7

    ; Exit
    mov eax, 1
    int 0x80
```

**Expected:** `ecx = 2`, `edx = 7`

---

### **B. Procedures are Defined Using Specific Syntax**

---

### 🧠 What is a Procedure?

A **procedure** (also called a subroutine or function) is a reusable block of code that performs a specific task and can be called from multiple places in a program.

In **assembly language**, procedures are defined with **labels** and use instructions like `call` and `ret`.

---

### 🛠 Syntax for Defining a Procedure

Here's the basic layout:

```nasm
; Procedure definition
MyProcedure:
    ; Procedure body
    ; Do something...
    ret        ; Return to caller
```

* The **label** (`MyProcedure:`) marks the start of the procedure.
* The **`ret`** instruction tells the CPU to go back to the instruction after `call`.

---

### ✅ Example: Basic Procedure Definition

```nasm
section .text
global _start

_start:
    call Greet        ; Call the procedure
    ; Exit program
    mov eax, 1
    int 0x80

; Subroutine
Greet:
    ; You could insert print instructions here
    ; Just a placeholder for now
    ret
```

In a more complete version, this procedure might output text or perform logic, and we would use `sys_write`.

---

### ⚙️ Notes

* **Procedures do not require arguments**, but can receive them via registers or the stack.
* They must **end with `ret`**, or the program will crash or jump to an unknown location.
* You can define multiple procedures in the same `.asm` file.

---

## **V. How Subroutines are Placed in Procedures**

### **C. The Call to a Procedure**

---

### 📞 What Does `call` Do?

The `call` instruction in assembly:

1. **Pushes the return address** (next instruction’s address) onto the stack
2. **Jumps** to the specified procedure label

This lets the program return to the correct point once the procedure finishes.

---

### 🔁 Returning with `ret`

The `ret` instruction:

* Pops the return address from the stack
* Jumps back to that address

Together, `call` and `ret` control program flow into and out of subroutines.

---

### 📌 Syntax

```nasm
call MyProcedure
...
MyProcedure:
    ; do something
    ret
```

---

### 🧪 Example: Calling a Procedure

```nasm
section .text
global _start

_start:
    call SayHello     ; jump to procedure
    call SayHello     ; call again
    mov eax, 1
    int 0x80

SayHello:
    ; You could insert sys_write to print text here
    ret
```

Even though the same procedure is called twice, each time it **returns to the correct spot**.

---

### 🔬 What's on the Stack?

Each `call` pushes a return address:

```
Stack (top):
[Return address 2]
[Return address 1]
```

Each `ret` pops the top address and returns there.

---

⬇️

### **D. What to Do When Procedures Are in an External Library**

---

### 📦 What is an External Procedure?

An **external procedure** is one that's defined outside your assembly file - usually in another `.obj`, `.dll`, or `.so` file (depending on your OS). You link to it during compilation.

Examples:

* C standard library functions like `printf`, `malloc`
* External assembly routines in other files

---

### 🔧 How to Use External Procedures in NASM (Linux Example)

#### 🪛 Step 1: Declare the external procedure

```nasm
extern printf
```

#### 🪛 Step 2: Link against its library (e.g., C library)

During linking, use:

```bash
ld -dynamic-linker /lib64/ld-linux-x86-64.so.2 -lc
```

#### 🧪 Minimal Example (uses C's `printf`)

```nasm
section .data
    msg db "Hello from printf!", 0

section .text
    extern printf
    global _start

_start:
    mov rdi, msg       ; First argument to printf
    xor rax, rax       ; printf is variadic, so clear RAX for 0 floats
    call printf

    ; Exit
    mov eax, 60
    xor edi, edi
    syscall
```

#### 🧩 Compile and Link

```bash
nasm -f elf64 extern_example.asm -o extern_example.o
gcc extern_example.o -o extern_example
./extern_example
```

> This links NASM output with the C runtime using `gcc`.

---

### 🧠 Summary

* Use `extern` to declare an external symbol.
* Link using `gcc` or `ld` with the correct libraries.
* External procedures let you **reuse system and C standard functions**.

---

⬇️

### **E. What is a Stack Frame and How Is It Used?**

---

### 🧱 What Is a Stack Frame?

A **stack frame** is a structured section of memory on the call stack that is allocated every time a **procedure (subroutine)** is called. It acts as a container for:

* The **return address**
* **Function arguments** passed on the stack
* **Saved register values** (especially `ebp`/`rbp`)
* **Local variables**

Each function has **its own stack frame**, which keeps its execution context separate and protected from other functions.

---

### 🧭 Why Use Stack Frames?

Without stack frames:

* It would be hard to track where to return after a procedure
* Temporary variables would overwrite each other
* Arguments and locals would collide

Stack frames bring **organization and predictability**, especially in nested function calls or recursion.

---

### 🧪 Anatomy of a Stack Frame (32-bit Example)

Let’s walk through what a stack frame typically looks like:

```
Stack (grows downward)

┌──────────────┐  ← higher memory
│ Return Addr  │  ← pushed by `call`
├──────────────┤
│ Saved EBP    │  ← previous frame pointer
├──────────────┤
│ Arguments    │  ← [ebp + 8], [ebp + 12], ...
├──────────────┤
│ Local Vars   │  ← [ebp - 4], [ebp - 8], ...
└──────────────┘  ← lower memory
```

---

### 🧰 How to Create and Use a Stack Frame

#### 🔹 Prologue (setup):

```nasm
push ebp          ; Save caller's base pointer
mov ebp, esp      ; Set base pointer to current stack top
sub esp, N        ; Reserve space for local variables (N = bytes)
```

#### 🔹 Access data:

* Arguments: `[ebp + 8]`, `[ebp + 12]`, ...
* Locals: `[ebp - 4]`, `[ebp - 8]`, ...

#### 🔹 Epilogue (restore and return):

```nasm
mov esp, ebp      ; Restore stack pointer
pop ebp           ; Restore caller's frame
ret               ; Return to caller
```

---

### 📌 Full Example (Add Two Arguments)

```nasm
section .text
global _start

extern printf

_start:
    push 3
    push 5
    call Add
    add esp, 8       ; Clean up stack after call

    ; Exit
    mov eax, 60
    xor edi, edi
    syscall

Add:
    push ebp
    mov ebp, esp

    mov eax, [ebp + 8]   ; First arg
    add eax, [ebp + 12]  ; Add second arg

    pop ebp
    ret
```

---

### 🧠 Key Takeaways

* `ebp` (or `rbp` in x86\_64) is used as a **stable reference point**.
* `esp` (or `rsp`) moves during function calls and `push`/`pop`, so it's unreliable for direct indexing.
* Stack frames are essential for **clean function call mechanics**, especially when dealing with **recursion**, **nested calls**, or **local variables**.

---

### **F. Performing Recursion**

---

### 🔁 What is Recursion?

**Recursion** is when a function **calls itself** to solve a smaller part of a problem until a base condition is met.

Assembly language supports recursion by leveraging:

* **The call stack** to manage multiple function contexts
* **Stack frames** to isolate each recursive call's data

---

### ⚙️ How It Works

Each recursive call:

1. Pushes a **new return address**
2. Pushes its **own arguments** and **local state**
3. **Returns** once the base case is met

The stack grows **deeper** with each call, then **shrinks** as calls return.

---

### 🧠 Recursive Structure in Assembly (x86 Example)

#### Recursion Skeleton:

```nasm
RecursiveFunc:
    push ebp
    mov ebp, esp

    ; base case check
    ; recursive call

    pop ebp
    ret
```

---

### 🧪 Example: Recursive Factorial in NASM

```nasm
section .text
global _start

_start:
    mov eax, 5       ; Calculate factorial(5)
    call factorial

    ; Exit program
    mov ebx, 0
    mov eax, 1
    int 0x80

factorial:
    push ebp
    mov ebp, esp

    cmp eax, 1        ; base case
    jbe .base_case

    push eax          ; save eax
    dec eax
    call factorial    ; recursive call
    pop ebx           ; restore original eax
    imul eax, ebx     ; multiply result

    jmp .done

.base_case:
    mov eax, 1

.done:
    pop ebp
    ret
```

---

### 📌 Explanation

* `cmp eax, 1` checks if the base case is reached.
* Each call to `factorial` pushes its own value onto the stack.
* Once recursion unwinds, the results are multiplied in reverse order.

---

### ⚠️ Notes

* **Recursive assembly** must manage the stack carefully.
* Always **preserve registers** and maintain clean stack frames.
* Without a base case, recursion can crash the program via **stack overflow**.

---

⬇️

### **G. `INVOKE`: The Powerful Procedure Tool in 32-bit x86 Assembler**

---

### 🧠 What is `INVOKE`?

`INVOKE` is a **high-level macro-like directive** available in some 32-bit x86 assemblers (like **MASM** and **TASM**), used to **call procedures with arguments** more cleanly and automatically.

It simplifies:

* Pushing arguments in the correct order
* Calling the procedure
* Cleaning up the stack afterward

> 📝 Note: `INVOKE` is not available in NASM or GAS — only in macro assemblers like MASM.

---

### 🔍 Syntax

```asm
INVOKE ProcedureName, arg1, arg2, ...
```

This:

* Pushes each argument **right-to-left** (as per x86 calling convention)
* Calls the procedure

---

### 🧪 Example in MASM:

```asm
INVOKE MessageBoxA, NULL, OFFSET msgText, OFFSET msgTitle, MB_OK
```

This is equivalent to:

```asm
push MB_OK
push OFFSET msgTitle
push OFFSET msgText
push NULL
call MessageBoxA
```

---

### ✅ Benefits

| Feature           | Traditional ASM | With `INVOKE` |
| ----------------- | --------------- | ------------- |
| Argument ordering | Manual          | Handled       |
| Readability       | Low             | High          |
| Error-prone       | High            | Lower         |

---

### ⚠️ Limitations

* Only available in **high-level assemblers** (MASM, TASM)
* Not portable across NASM or Linux environments
* Requires **prototypes** to be declared for the procedures

---

### 📚 Summary

* `INVOKE` makes procedure calls look more like high-level languages (C, Pascal).
* Simplifies procedure calling and improves readability.
* Best used in **Windows programming with MASM**.

---

## ✅ **Summary of Section V: How Subroutines are Placed in Procedures**

---

### **A. How Stack Operations Work**

* The **stack** is used to save register values and pass parameters.
* **`push`** and **`pop`** manage data on the stack in LIFO order.
* Temporary data is stored during procedure calls to maintain program state.

---

### **B. Procedures are Defined Using Specific Syntax**

* Procedures are labeled blocks of code ending in **`ret`**.
* They can be reused and called multiple times with `call`.

---

### **C. The Call to a Procedure**

* The **`call`** instruction pushes the return address and jumps to the procedure.
* **`ret`** pops the return address and returns control to the caller.

---

### **D. What to Do When Procedures Are in an External Library**

* Use **`extern`** to declare functions like `printf`.
* Link with external libraries using `gcc` or `ld`.
* Enables access to C functions and system-level routines.

---

### **E. What Is a Stack Frame and How Is It Used**

* A **stack frame** organizes procedure data like return addresses, parameters, and locals.
* Built using `push ebp`, `mov ebp, esp`, and cleaned up with `pop ebp`.
* Essential for recursion and nested function calls.

---

### **F. Performing Recursion**

* Recursion involves a function calling itself.
* The stack tracks each call with its own arguments and return address.
* Base case and stack cleanup are critical to avoid overflow.

---

### **G. `INVOKE`, the Powerful Procedure Tool**

* Available in MASM, `INVOKE` automates pushing arguments and calling procedures.
* Improves readability and reduces manual errors.
* Not available in NASM or Linux tools.

---